In [ ]:
import pandas as pd
import numpy as np


LISTINGS_PATH = "./listings.csv"
CALENDAR_PATH = "./calendar.csv"
REVIEWS_PATH = "./reviews.csv"
LISTINGS_ID_COL = "id"
CALENDAR_ID_COL = "listing_id"
REVIEWS_ID_COL = "listing_id"


print(f"Loading listings from: {LISTINGS_PATH}")
df_listings = pd.read_csv(LISTINGS_PATH, low_memory=False)

print(f"Loading calendar from: {CALENDAR_PATH}")

df_calendar = pd.read_csv(CALENDAR_PATH)
if "date" in df_calendar.columns:
    df_calendar["date"] = pd.to_datetime(df_calendar["date"], errors="coerce")

print(f"Loading reviews from: {REVIEWS_PATH}")

df_reviews = pd.read_csv(REVIEWS_PATH)
if "date" in df_reviews.columns:
    df_reviews["date"] = pd.to_datetime(df_reviews["date"], errors="coerce")

print("\n--- Initial Data Shapes ---")
print(f"Listings: {df_listings.shape}")
print(f"Calendar: {df_calendar.shape}")
print(f"Reviews: {df_reviews.shape}")


print("\n--- Listing ID Analysis ---")

id_stats = {}


def calculate_id_stats(df, name, id_col):
    if id_col not in df.columns:
        print(f"Warning: '{id_col}' column not found in {name}.")
        return {
            "Total Rows": len(df),
            f"Rows with non-null {id_col}": 0,
            f"Unique {id_col}s": 0,
            f"% Non-Null {id_col}": 0,
        }

    total_rows = len(df)
    non_null_ids = df[id_col].notna().sum()
    unique_ids = df[id_col].nunique()
    stats = {
        "Total Rows": total_rows,
        f"Rows with non-null {id_col}": non_null_ids,
        f"Unique {id_col}s": unique_ids,
        f"% Non-Null {id_col}": (
            round((non_null_ids / total_rows) * 100, 2) if total_rows > 0 else 0
        ),
    }
    return stats


id_stats["listings"] = calculate_id_stats(df_listings, "listings", LISTINGS_ID_COL)
id_stats["calendar"] = calculate_id_stats(df_calendar, "calendar", CALENDAR_ID_COL)
id_stats["reviews"] = calculate_id_stats(df_reviews, "reviews", REVIEWS_ID_COL)

stats_df = pd.DataFrame(id_stats).T
print(stats_df)


print("\n--- Merge Simulation (Listings INNER JOIN Calendar) ---")


if LISTINGS_ID_COL in df_listings.columns and CALENDAR_ID_COL in df_calendar.columns:
    listings_ids = (
        pd.to_numeric(df_listings[LISTINGS_ID_COL], errors="coerce").dropna().unique()
    )
    calendar_ids = (
        pd.to_numeric(df_calendar[CALENDAR_ID_COL], errors="coerce").dropna().unique()
    )

    df_listings_ids = pd.DataFrame({CALENDAR_ID_COL: listings_ids})
    df_calendar_ids = pd.DataFrame({CALENDAR_ID_COL: calendar_ids})

    merged_df = pd.merge(
        df_listings_ids, df_calendar_ids, on=CALENDAR_ID_COL, how="inner"
    )

    merged_unique_ids = merged_df[CALENDAR_ID_COL].nunique()
    listings_unique_ids = len(listings_ids)
    calendar_unique_ids = len(calendar_ids)

    print(
        f"Unique valid listing_ids in listings (col '{LISTINGS_ID_COL}'): {listings_unique_ids}"
    )
    print(
        f"Unique valid listing_ids in calendar (col '{CALENDAR_ID_COL}'): {calendar_unique_ids}"
    )
    print(
        f"Unique listing_ids present in BOTH listings and calendar (after cleaning): {merged_unique_ids}"
    )

    if listings_unique_ids > 0:
        percentage_retained = round((merged_unique_ids / listings_unique_ids) * 100, 2)
        print(
            f"Percentage of unique listings retained after merge: {percentage_retained}%"
        )
    else:
        print(
            "Cannot calculate retention percentage, no valid unique listings found in listings df."
        )
else:
    print(
        f"Skipping merge simulation: ID column missing ('{LISTINGS_ID_COL}' in listings or '{CALENDAR_ID_COL}' in calendar)."
    )

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


if "df_listings" not in locals():
    print("Error: df_listings not found. Please run the previous cell first.")
else:
    NEIGHBORHOOD_COL = "neighbourhood_cleansed"

    if NEIGHBORHOOD_COL not in df_listings.columns:
        print(f"Error: Column '{NEIGHBORHOOD_COL}' not found in listings dataframe.")
    else:
        total_listings = len(df_listings)
        missing_neighborhoods = df_listings[NEIGHBORHOOD_COL].isna().sum()
        percentage_missing = (
            round((missing_neighborhoods / total_listings) * 100, 2)
            if total_listings > 0
            else 0
        )

        print(f"\n--- Missing Neighborhood Analysis ('{NEIGHBORHOOD_COL}') ---")
        print(f"Total listings: {total_listings}")
        print(f"Listings missing '{NEIGHBORHOOD_COL}': {missing_neighborhoods}")
        print(f"Percentage missing: {percentage_missing}%")

        print(f"\nPlotting distribution of listings per '{NEIGHBORHOOD_COL}'...")

        neighborhood_counts = (
            df_listings[NEIGHBORHOOD_COL].fillna("Missing").value_counts()
        )

        plt.figure(figsize=(12, 8))
        sns.barplot(
            y=neighborhood_counts.index, x=neighborhood_counts.values, palette="viridis"
        )

        plt.title(f'Number of Listings per Neighborhood ("{NEIGHBORHOOD_COL}")')
        plt.xlabel("Number of Listings")
        plt.ylabel("Neighborhood")
        plt.tight_layout()
        plt.show()

        print("\nTop 10 Neighborhoods by Listing Count (including Missing):")
        print(neighborhood_counts.head(10))

        if "latitude" in df_listings.columns and "longitude" in df_listings.columns:
            missing_lat = df_listings["latitude"].isna().sum()
            missing_lon = df_listings["longitude"].isna().sum()
            print("\n--- Lat/Lon Completeness Check ---")
            print(
                f"Missing latitude: {missing_lat} ({round(missing_lat * 100 / total_listings, 2)}%)"
            )
            print(
                f"Missing longitude: {missing_lon} ({round(missing_lon * 100 / total_listings, 2)}%)"
            )

            if missing_neighborhoods > 0:
                missing_neigh_df = df_listings[df_listings[NEIGHBORHOOD_COL].isna()]
                missing_lat_in_missing_neigh = missing_neigh_df["latitude"].isna().sum()
                missing_lon_in_missing_neigh = (
                    missing_neigh_df["longitude"].isna().sum()
                )
                print(
                    f"  - Among listings missing neighborhood: {missing_lat_in_missing_neigh} missing latitude ({round(missing_lat_in_missing_neigh * 100 / missing_neighborhoods, 2)}%)"
                )
                print(
                    f"  - Among listings missing neighborhood: {missing_lon_in_missing_neigh} missing longitude ({round(missing_lon_in_missing_neigh * 100 / missing_neighborhoods, 2)}%)"
                )
            else:
                print("No listings are missing neighborhood data.")

        else:
            print("\nLatitude/Longitude columns not found for imputation check.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


print("\n--- Geospatial Distribution of Missing Neighborhoods ---")

if (
    NEIGHBORHOOD_COL in df_listings.columns
    and "latitude" in df_listings.columns
    and "longitude" in df_listings.columns
):
    listings_with_coords = df_listings.dropna(subset=["latitude", "longitude"])
    listings_missing_neigh = listings_with_coords[
        listings_with_coords[NEIGHBORHOOD_COL].isna()
    ]
    listings_valid_neigh = listings_with_coords[
        listings_with_coords[NEIGHBORHOOD_COL].notna()
    ]

    num_missing_plot = len(listings_missing_neigh)

    if num_missing_plot > 0:
        print(
            f"Plotting {num_missing_plot} listings with coordinates but missing neighborhood..."
        )

        plt.figure(figsize=(10, 10))

        sns.scatterplot(
            data=listings_valid_neigh,
            x="longitude",
            y="latitude",
            color="blue",
            alpha=0.5,
            s=15,
            label="Valid Neighborhood",
        )

        sns.scatterplot(
            data=listings_missing_neigh,
            x="longitude",
            y="latitude",
            color="red",
            alpha=0.5,
            s=15,
            label="Missing Neighborhood",
        )

        plt.title("Spatial Distribution of Listings with Missing Neighborhood Data")
        plt.xlabel("Longitude")
        plt.ylabel("Latitude")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    else:
        print(
            "No listings found with coordinates but missing neighborhood data to plot."
        )

else:
    print(
        "Skipping geospatial plot: Required columns ('neighbourhood_cleansed', 'latitude', 'longitude') not available."
    )

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import timedelta


print("--- Cleaning and Analyzing Calendar Price ---")


def clean_price(price_str):
    if pd.isna(price_str):
        return np.nan

    if isinstance(price_str, str):
        price_clean = re.sub(r"[^0-9.]", "", price_str)
        try:
            return float(price_clean)
        except ValueError:
            return np.nan
    else:
        return price_str


if df_calendar["price"].dtype == "object":
    print("Cleaning price column...")
    df_calendar["price_numeric"] = df_calendar["price"].apply(clean_price)
else:
    print("Price column already numeric.")
    df_calendar["price_numeric"] = df_calendar["price"]


price_stats = df_calendar["price_numeric"].describe()
print("\nPrice Statistics:")
print(price_stats)


q1 = price_stats["25%"]
q3 = price_stats["75%"]
iqr = q3 - q1
price_lower_bound = q1 - 1.5 * iqr
price_upper_bound = q3 + 1.5 * iqr
price_outliers = df_calendar[
    (df_calendar["price_numeric"] < price_lower_bound)
    | (df_calendar["price_numeric"] > price_upper_bound)
]

print(
    f"\nPrice Outliers Detected: {len(price_outliers)} rows ({round(len(price_outliers) * 100 / len(df_calendar), 2)}% of total)"
)
print(
    f"Price Range for non-outliers: £{price_lower_bound:.2f} to £{price_upper_bound:.2f}"
)


plt.figure(figsize=(10, 6))
sns.histplot(df_calendar["price_numeric"].dropna(), bins=50, kde=True)
plt.title("Distribution of Daily Prices (with outliers)")
plt.xlabel("Price")
plt.ylabel("Count")
plt.xlim(0, price_stats["75%"] * 3)
plt.axvline(
    price_upper_bound,
    color="red",
    linestyle="--",
    label=f"Upper Bound (£{price_upper_bound:.2f})",
)
plt.legend()
plt.show()


print("\n--- Availability Flag Analysis ---")


available_values = df_calendar["available"].value_counts(dropna=False)
print("Values in 'available' column:")
print(available_values)


missing_available = df_calendar["available"].isna().sum()
missing_available_pct = round(missing_available * 100 / len(df_calendar), 2)
print(f"Missing 'available' values: {missing_available} ({missing_available_pct}%)")


if "available_numeric" not in df_calendar.columns:
    df_calendar["available_numeric"] = df_calendar["available"].map({"t": 1, "f": 0})


print("\n--- Observation Period Analysis ---")


if df_calendar["date"].dtype == "<M8[ns]":
    date_ranges = (
        df_calendar.groupby("listing_id")["date"].agg(["min", "max"]).reset_index()
    )
    date_ranges["observation_days"] = (
        date_ranges["max"] - date_ranges["min"]
    ).dt.days + 1

    obs_days_stats = date_ranges["observation_days"].describe()
    print("\nObservation Period Statistics (days):")
    print(obs_days_stats)

    plt.figure(figsize=(10, 6))
    sns.histplot(date_ranges["observation_days"], bins=50, kde=True)
    plt.title("Distribution of Observation Period Length per Listing")
    plt.xlabel("Observation Period (days)")
    plt.ylabel("Number of Listings")
    plt.axvline(365, color="red", linestyle="--", label="Full Year (365 days)")
    plt.legend()
    plt.show()

    short_obs = date_ranges[date_ranges["observation_days"] < 30]
    print(
        f"\nListings with very short observation periods (<30 days): {len(short_obs)} ({round(len(short_obs) * 100 / len(date_ranges), 2)}% of listings)"
    )

    listing_counts = (
        df_calendar.groupby("listing_id").size().reset_index(name="data_points")
    )
    merged_obs = pd.merge(date_ranges, listing_counts, on="listing_id")

    merged_obs["points_per_day"] = (
        merged_obs["data_points"] / merged_obs["observation_days"]
    )
    print("\nData Points per Day Statistics:")
    print(merged_obs["points_per_day"].describe())

    complete_coverage = merged_obs[merged_obs["points_per_day"] >= 0.95].shape[0]
    print(
        f"Listings with near-complete daily coverage (≥0.95 points/day): {complete_coverage} ({round(complete_coverage * 100 / len(merged_obs), 2)}%)"
    )

    if "available_numeric" in df_calendar.columns:
        print("\n--- Sample Occupancy Rate Calculation ---")
        occupancy = (
            df_calendar.groupby("listing_id")["available_numeric"]
            .agg(
                total_days="count",
                available_days=lambda x: (x == 1).sum(),
                booked_days=lambda x: (x == 0).sum(),
            )
            .reset_index()
        )

        occupancy["occupancy_rate"] = occupancy["booked_days"] / occupancy["total_days"]
        print("\nOccupancy Rate Statistics:")
        print(occupancy["occupancy_rate"].describe())

        occupancy_with_days = pd.merge(occupancy, date_ranges, on="listing_id")

        plt.figure(figsize=(10, 6))
        plt.scatter(
            occupancy_with_days["observation_days"],
            occupancy_with_days["occupancy_rate"],
            alpha=0.5,
            s=10,
        )
        plt.title("Occupancy Rate vs. Observation Period")
        plt.xlabel("Observation Period (days)")
        plt.ylabel("Occupancy Rate")
        plt.grid(True, alpha=0.3)
        plt.show()

        corr = (
            occupancy_with_days[["observation_days", "occupancy_rate"]]
            .corr()
            .iloc[0, 1]
        )
        print(
            f"\nCorrelation between observation period and occupancy rate: {corr:.4f}"
        )

        if abs(corr) > 0.1:
            print(
                "SIGNIFICANT CORRELATION: Observation period length may be biasing occupancy calculations."
            )
        else:
            print(
                "Low correlation: Observation period length does not appear to significantly bias occupancy calculations."
            )
else:
    print("Date column not in datetime format. Skipping observation period analysis.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


print("\n--- Distribution of Calculated Occupancy Rates ---")


if (
    "occupancy" in locals()
    and isinstance(occupancy, pd.DataFrame)
    and "occupancy_rate" in occupancy.columns
):
    plt.figure(figsize=(10, 6))
    sns.histplot(occupancy["occupancy_rate"], bins=50, kde=True)
    plt.title("Distribution of Calculated Occupancy Rate per Listing")
    plt.xlabel("Occupancy Rate")
    plt.ylabel("Number of Listings")
    plt.grid(True, alpha=0.3)
    plt.show()

    zero_occ = (occupancy["occupancy_rate"] == 0).sum()
    full_occ = (occupancy["occupancy_rate"] == 1).sum()
    print(f"Listings with 0% calculated occupancy: {zero_occ}")
    print(f"Listings with 100% calculated occupancy: {full_occ}")

else:
    print(
        "Skipping occupancy distribution plot: 'occupancy' DataFrame not found or missing 'occupancy_rate' column."
    )
    print("Ensure the previous parts of Task 3 executed correctly.")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


KEY_LISTING_FEATURES = [
    "id",
    "neighbourhood_cleansed",
    "property_type",
    "room_type",
    "accommodates",
    "bathrooms",
    "bedrooms",
    "beds",
    "price",
    "latitude",
    "longitude",
    "review_scores_rating",
    "review_scores_accuracy",
    "review_scores_cleanliness",
    "review_scores_checkin",
    "review_scores_communication",
    "review_scores_location",
    "review_scores_value",
    "number_of_reviews",
    "reviews_per_month",
    "availability_365",
]


key_features_df = df_listings[KEY_LISTING_FEATURES].copy()


print("--- Missing Value Analysis ---")


missing_percentages = key_features_df.isna().mean() * 100
missing_percentages = missing_percentages.sort_values(ascending=False)


missing_df = pd.DataFrame(
    {
        "Feature": missing_percentages.index,
        "Missing %": missing_percentages.values,
        "Non-Missing Count": key_features_df.shape[0] - key_features_df.isna().sum(),
        "Total Count": key_features_df.shape[0],
    }
)
print("\nMissing Values by Feature:")
print(missing_df)


plt.figure(figsize=(12, 8))
sns.barplot(y=missing_percentages.index, x=missing_percentages, palette="viridis")
plt.title("Percentage of Missing Values by Feature")
plt.xlabel("Missing Percentage (%)")
plt.ylabel("Feature")
plt.tight_layout()
plt.show()


print("\n--- Numerical Feature Distributions ---")


numerical_features = key_features_df.select_dtypes(
    include=["float64", "int64"]
).columns.tolist()


if "price" in key_features_df.columns and key_features_df["price"].dtype == "object":

    def clean_listing_price(price_str):
        if pd.isna(price_str):
            return np.nan

        if isinstance(price_str, str):
            price_clean = re.sub(r"[^0-9.]", "", price_str)
            try:
                return float(price_clean)
            except ValueError:
                return np.nan
        else:
            return price_str

    key_features_df["price_numeric"] = key_features_df["price"].apply(
        clean_listing_price
    )
    if "price" in numerical_features:
        numerical_features.remove("price")
    numerical_features.append("price_numeric")


print("Plotting distributions for key numerical features...")
for column in [
    "accommodates",
    "bedrooms",
    "beds",
    "bathrooms",
    "review_scores_rating",
    "number_of_reviews",
    "reviews_per_month",
]:
    if column in numerical_features:
        plt.figure(figsize=(10, 6))
        sns.histplot(key_features_df[column].dropna(), kde=True)
        plt.title(f"Distribution of {column}")
        plt.xlabel(column)
        plt.ylabel("Count")
        plt.tight_layout()
        plt.show()

        print(f"\nStatistics for {column}:")
        print(key_features_df[column].describe())


if "price_numeric" in numerical_features:
    plt.figure(figsize=(10, 6))

    q3 = key_features_df["price_numeric"].quantile(0.75)
    sns.histplot(key_features_df["price_numeric"].dropna(), kde=True)
    plt.title("Distribution of Listing Price")
    plt.xlabel("Price")
    plt.ylabel("Count")
    plt.xlim(0, q3 * 3)
    plt.tight_layout()
    plt.show()

    print("\nStatistics for listing price:")
    print(key_features_df["price_numeric"].describe())


print("\n--- Categorical Feature Distributions ---")


categorical_features = ["property_type", "room_type"]
for feature in categorical_features:
    if feature in key_features_df.columns:
        value_counts = key_features_df[feature].value_counts().reset_index()
        value_counts.columns = [feature, "Count"]

        value_counts["Percentage"] = (
            value_counts["Count"] / value_counts["Count"].sum() * 100
        )

        print(f"\nValue counts for {feature}:")
        print(value_counts)

        top_n = min(15, len(value_counts))
        plt.figure(figsize=(12, 6))
        sns.barplot(
            data=value_counts.head(top_n), y=feature, x="Count", palette="viridis"
        )
        plt.title(f"Top {top_n} Categories for {feature}")
        plt.xlabel("Count")
        plt.ylabel(feature)
        plt.tight_layout()
        plt.show()


print("\n--- Correlation Analysis ---")


correlation_features = [f for f in numerical_features if f != "id"]
if correlation_features:
    corr_matrix = key_features_df[correlation_features].corr().round(2)

    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(
        corr_matrix,
        annot=True,
        mask=mask,
        cmap="coolwarm",
        center=0,
        linewidths=0.5,
        cbar_kws={"shrink": 0.8},
    )
    plt.title("Correlation Matrix of Numerical Features")
    plt.tight_layout()
    plt.show()

    strong_correlations = []
    for i in range(len(correlation_features)):
        for j in range(i + 1, len(correlation_features)):
            if abs(corr_matrix.iloc[i, j]) > 0.5:
                strong_correlations.append(
                    {
                        "Feature 1": correlation_features[i],
                        "Feature 2": correlation_features[j],
                        "Correlation": corr_matrix.iloc[i, j],
                    }
                )

    if strong_correlations:
        print("\nStrong Correlations (|correlation| > 0.5):")
        strong_corr_df = pd.DataFrame(strong_correlations)
        print(strong_corr_df.sort_values("Correlation", key=abs, ascending=False))
    else:
        print("\nNo strong correlations found between numerical features.")

In [ ]:
import matplotlib.pyplot as plt


print("\n--- Advanced Missing Value Patterns (missingno) ---")

try:
    import missingno as msno

    print("Visualizing missing value matrix...")
    msno.matrix(key_features_df)
    plt.title("Missing Value Matrix (Listings Key Features)", fontsize=16)
    plt.show()

    print("\nVisualizing missing value heatmap...")
    msno.heatmap(key_features_df)
    plt.title("Missing Value Correlation Heatmap (Listings Key Features)", fontsize=16)
    plt.show()

except ImportError:
    print("\nSkipping missingno plots: library 'missingno' not installed.")
    print("Install using: pip install missingno")
except Exception as e:
    print(f"\nAn error occurred during missingno plotting: {e}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


print("\n--- Distribution of Occupancy Rate Discrepancy ---")


if (
    "avail_comparison" in locals()
    and isinstance(avail_comparison, pd.DataFrame)
    and "abs_diff" in avail_comparison.columns
):
    plt.figure(figsize=(10, 6))
    sns.histplot(avail_comparison["abs_diff"], bins=40, kde=False)
    plt.title("Distribution of Absolute Difference between Occupancy Measures")
    plt.xlabel("Absolute Difference (Implied vs. Calculated Occupancy Rate)")
    plt.ylabel("Number of Listings")
    plt.grid(True, alpha=0.3)
    plt.show()

    low_diff = (avail_comparison["abs_diff"] < 0.05).sum()
    low_diff_pct = round(low_diff * 100 / len(avail_comparison), 2)
    print(
        f"Listings with very low discrepancy (< 0.05 or 5%): {low_diff} ({low_diff_pct}%)"
    )
else:
    print(
        "Skipping discrepancy histogram: 'avail_comparison' DataFrame not found or missing 'abs_diff' column."
    )
    print("Ensure the previous parts of Task 5 executed correctly.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


print("--- Review Score Consistency Check ---")


if "numerical_review" in df_reviews.columns and REVIEWS_ID_COL in df_reviews.columns:
    avg_reviews = (
        df_reviews.dropna(subset=[REVIEWS_ID_COL, "numerical_review"])
        .groupby(REVIEWS_ID_COL)["numerical_review"]
        .agg(["mean", "count"])
        .reset_index()
    )
    avg_reviews.columns = [REVIEWS_ID_COL, "avg_numerical_review", "review_count"]

    if (
        "review_scores_rating" in df_listings.columns
        and LISTINGS_ID_COL in df_listings.columns
    ):
        listings_reviews = df_listings[[LISTINGS_ID_COL, "review_scores_rating"]].copy()
        listings_reviews.rename(columns={LISTINGS_ID_COL: REVIEWS_ID_COL}, inplace=True)

        review_comparison = pd.merge(
            listings_reviews, avg_reviews, on=REVIEWS_ID_COL, how="inner"
        )

        print(f"Successfully merged reviews for {len(review_comparison)} listings")
        print("\nReview scores statistics:")
        review_stats = review_comparison[
            ["review_scores_rating", "avg_numerical_review"]
        ].describe()
        print(review_stats)

        review_comparison["abs_diff"] = abs(
            review_comparison["review_scores_rating"]
            - review_comparison["avg_numerical_review"] * 2
        )
        review_corr = (
            review_comparison[["review_scores_rating", "avg_numerical_review"]]
            .corr()
            .iloc[0, 1]
        )

        print(
            f"\nCorrelation between listing review score and average numerical review: {review_corr:.4f}"
        )
        print("\nAbsolute difference statistics:")
        print(review_comparison["abs_diff"].describe())

        large_diff_count = (review_comparison["abs_diff"] > 1).sum()
        large_diff_pct = round(large_diff_count * 100 / len(review_comparison), 2)
        print(
            f"\nListings with large review score discrepancy (>1 point): {large_diff_count} ({large_diff_pct}%)"
        )

        plt.figure(figsize=(10, 8))
        plt.scatter(
            review_comparison["avg_numerical_review"] * 2,
            review_comparison["review_scores_rating"],
            alpha=0.5,
            s=10,
        )
        plt.plot([0, 10], [0, 10], color="red", linestyle="--")
        plt.title("Listing Review Score vs Average Numerical Review")
        plt.xlabel("Average Numerical Review (scaled to 0-10 range)")
        plt.ylabel("Review Score Rating (0-10)")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

        reliable_reviews = review_comparison[
            review_comparison["review_count"] >= 5
        ].copy()
        if len(reliable_reviews) > 0:
            reliable_corr = (
                reliable_reviews[["review_scores_rating", "avg_numerical_review"]]
                .corr()
                .iloc[0, 1]
            )
            print(
                f"\nCorrelation for listings with 5+ reviews ({len(reliable_reviews)} listings): {reliable_corr:.4f}"
            )

    else:
        print(
            "Unable to check review score consistency: 'review_scores_rating' not found in listings data"
        )
else:
    print(
        "Unable to calculate average numerical reviews: required columns not found in reviews data"
    )


print("\n--- Availability vs. Occupancy Consistency Check ---")


occupancy_from_calendar_exists = False
try:
    if "occupancy" not in locals() and "available_numeric" in df_calendar.columns:
        occupancy = (
            df_calendar.groupby(CALENDAR_ID_COL)["available_numeric"]
            .agg(
                total_days="count",
                available_days=lambda x: (x == 1).sum(),
                booked_days=lambda x: (x == 0).sum(),
            )
            .reset_index()
        )

        occupancy["occupancy_rate"] = occupancy["booked_days"] / occupancy["total_days"]
        occupancy_from_calendar_exists = True
    elif "occupancy" in locals():
        occupancy_from_calendar_exists = True
except Exception as e:
    print(f"Error calculating occupancy: {e}")
    occupancy_from_calendar_exists = False


if "availability_365" in df_listings.columns and LISTINGS_ID_COL in df_listings.columns:
    listings_avail = df_listings[[LISTINGS_ID_COL, "availability_365"]].copy()

    listings_avail["availability_rate"] = listings_avail["availability_365"] / 365

    if occupancy_from_calendar_exists:
        listings_avail.rename(columns={LISTINGS_ID_COL: CALENDAR_ID_COL}, inplace=True)

        avail_comparison = pd.merge(
            listings_avail, occupancy, on=CALENDAR_ID_COL, how="inner"
        )

        print(
            f"Successfully merged availability data for {len(avail_comparison)} listings"
        )

        avail_comparison["implied_occupancy"] = (
            1 - avail_comparison["availability_rate"]
        )

        avail_comparison["abs_diff"] = abs(
            avail_comparison["implied_occupancy"] - avail_comparison["occupancy_rate"]
        )
        avail_corr = (
            avail_comparison[["implied_occupancy", "occupancy_rate"]].corr().iloc[0, 1]
        )

        print(
            f"\nCorrelation between implied occupancy and calculated occupancy: {avail_corr:.4f}"
        )
        print("\nAbsolute difference statistics:")
        print(avail_comparison["abs_diff"].describe())

        large_diff_count = (avail_comparison["abs_diff"] > 0.2).sum()
        large_diff_pct = round(large_diff_count * 100 / len(avail_comparison), 2)
        print(
            f"\nListings with large occupancy discrepancy (>20%): {large_diff_count} ({large_diff_pct}%)"
        )

        plt.figure(figsize=(10, 8))
        plt.scatter(
            avail_comparison["implied_occupancy"],
            avail_comparison["occupancy_rate"],
            alpha=0.5,
            s=10,
        )
        plt.plot([0, 1], [0, 1], color="red", linestyle="--")
        plt.title("Implied Occupancy (from availability_365) vs Calculated Occupancy")
        plt.xlabel("Implied Occupancy Rate (1 - availability_365/365)")
        plt.ylabel("Calculated Occupancy Rate (from calendar)")
        plt.grid(True, alpha=0.3)
        plt.xlim(0, 1)
        plt.ylim(0, 1)
        plt.tight_layout()
        plt.show()

    else:
        print(
            "Unable to compare with calendar occupancy - occupancy data not calculated"
        )
else:
    print(
        "Unable to check availability consistency: 'availability_365' not found in listings data"
    )